In [1]:
import $file.^.testgeninit, testgeninit._
import $file.^.sparkinit, sparkinit._
import $file.^.pathinit, pathinit._
import $file.^.cpinit, cpinit._
import ss.implicits._
import org.apache.spark.sql.functions._
import java.nio.file.Paths
import com.relatedsciences.opentargets.etl.pipeline.SparkImplicits._
import scala.io.Source

Loading spark-stubs
Creating SparkSession


Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
19/12/12 21:36:53 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


import $file.$            , testgeninit._

import $file.$          , sparkinit._

import $file.$         , pathinit._

import $file.$       , cpinit._

import ss.implicits._

import org.apache.spark.sql.functions._

import java.nio.file.Paths

import com.relatedsciences.opentargets.etl.pipeline.SparkImplicits._

import scala.io.Source

In [2]:
// Load raw evidence extract 
val df = ss.read.json(EXTRACT_DIR.resolve("evidence_raw.json").toString)

json at cmd1.sc:1

9 / 9

19/12/12 21:37:17 WARN Utils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.debug.maxToStringFields' in SparkEnv.conf.


df: org.apache.spark.sql.package.DataFrame = [_corrupt_record: string, access_level: string ... 10 more fields]

In [3]:
val normalize = $"type" === "genetic_association" && $"evidence.gene2variant".isNotNull
def getECOScores: Map[String, Double] = {
    Source.fromURL("https://storage.googleapis.com/open-targets-data-releases/19.11/input/annotation-files/eco_scores-2019-10-31.tsv")
      .mkString
        .split("\n")
        .map(_.split("\t"))
        .map(r => r(0) -> r(2).toDouble).toMap
}
val ecolkp = typedLit(getECOScores)

normalize: org.apache.spark.sql.Column = ((type = genetic_association) AND (evidence.gene2variant IS NOT NULL))
defined function getECOScores
ecolkp: org.apache.spark.sql.Column = keys: [http://purl.obolibrary.org/obo/SO_0001060,http://purl.obolibrary.org/obo/SO_0001894,http://purl.obolibrary.org/obo/SO_0001627,http://purl.obolibrary.org/obo/SO_0001575,http://purl.obolibrary.org/obo/SO_0001566,http://purl.obolibrary.org/obo/SO_0001623,http://purl.obolibrary.org/obo/SO_0001782,http://purl.obolibrary.org/obo/SO_0001819,http://purl.obolibrary.org/obo/ECO_0000205,http://purl.obolibrary.org/obo/SO_0001589,http://purl.obolibrary.org/obo/SO_0001583,http://purl.obolibrary.org/obo/SO_0001792,http://purl.obolibrary.org/obo/SO_0001893,http://purl.obolibrary.org/obo/SO_0001587,http://purl.obolibrary.org/obo/SO_0001628,http://purl.obolibrary.org/obo/SO_0001567,http://purl.obolibrary.org/obo/SO_0001619,http://purl.obolibrary.org/obo/SO_0001574,http://purl.obolibrary.org/obo/SO_0001578,http://purl.ob

In [20]:
// Override resources scores only for ECO codes that are in the static score mapping (i.e. "enforce" them
// when known, otherwise leave them alone)
val dfe = df
  // Extract current resource score
  .withColumn("resource_score_original", when(normalize,
    coalesce($"evidence.gene2variant.resource_score.value", $"evidence.resource_score.value")
  ))
  // Get ECO URI for evidence
  .withColumn("resource_score_eco_uri", when(normalize,
    coalesce($"evidence.gene2variant.functional_consequence", $"evidence.evidence_codes".getItem(0))
  ))
  // Get expected score based on static URI -> score map
  .withColumn("resource_score_expected", when(normalize, ecolkp($"resource_score_eco_uri")))
  // Add flag for whether or not this "enforcement" is applicable
  .withColumn("resource_score_enforced_by_eco_code", $"resource_score_expected".isNotNull)
  // Add flag to indicate whether an update was actually necessary
      // Add flag to indicate what the value was changed to, if any change occurs
  .withColumn("resource_score_enforced_update",
    when($"resource_score_enforced_by_eco_code",
        when($"resource_score_original".isNull, lit("value_null"))
        .when($"resource_score_original" =!= $"resource_score_expected", lit("value_unequal"))
        .otherwise(lit("value_equal"))
    )
  )
  // Leave the resource score as is unless an override was applicable
  // .withColumn("evidence.gene2variant.resource_score",
  .withColumn("resource_score_new",
    when($"resource_score_enforced_by_eco_code", struct(
      $"evidence.gene2variant.resource_score.method".as("method"),
      lit("probability").as("type"),
      $"resource_score_expected".as("value")
    ))
    .otherwise($"evidence.gene2variant.resource_score")
  )
  // Drop some intermediate fields, move others to retained context
  .drop("resource_score_original", "resource_score_eco_uri", "resource_score_expected")
  .appendStruct("context", "resource_score_enforced_by_eco_code", "resource_score_enforced_update")

dfe: org.apache.spark.sql.package.DataFrame = [_corrupt_record: string, access_level: string ... 12 more fields]

In [21]:
dfe.groupBy("context.resource_score_enforced_by_eco_code", "type").count.show

show at cmd20.sc:1

9 / 9

show at cmd20.sc:1

1 / 1

+-----------------------------------+-------------------+------+
|resource_score_enforced_by_eco_code|               type| count|
+-----------------------------------+-------------------+------+
|                              false|       animal_model|502550|
|                              false|         known_drug|400440|
|                              false|     rna_expression|213490|
|                               true|genetic_association|400212|
|                              false|   somatic_mutation|107051|
|                              false|genetic_association|     8|
|                              false|   affected_pathway| 88291|
|                              false| genetic_literature| 17525|
|                              false|         literature|  1000|
|                              false|               null|     1|
+-----------------------------------+-------------------+------+



In [22]:
dfe.groupBy(
    $"context.resource_score_enforced_by_eco_code",
    $"context.resource_score_enforced_update"
).count.show

show at cmd21.sc:3

9 / 9

show at cmd21.sc:3

1 / 1

+-----------------------------------+------------------------------+-------+
|resource_score_enforced_by_eco_code|resource_score_enforced_update|  count|
+-----------------------------------+------------------------------+-------+
|                              false|                          null|1330356|
|                               true|                    value_null| 296855|
|                               true|                 value_unequal| 103357|
+-----------------------------------+------------------------------+-------+



In [23]:
// The values are unequal b/c they come in as "pvalue" type, not "probability"
dfe.filter($"context.resource_score_enforced_update" === "value_unequal")
    .select(
        "evidence.gene2variant.functional_consequence",
        "evidence.gene2variant.resource_score.*", 
        "evidence.evidence_codes",
        "target.id", "disease.id"
    )
    .show(5, false)

show at cmd22.sc:8

1 / 1

show at cmd22.sc:8

4 / 4

+-----------------------------------------+-------------------------------+------+------+--------------+----------------------------------------------+-----------------------------------------+
|functional_consequence                   |method                         |type  |value |evidence_codes|id                                            |id                                       |
+-----------------------------------------+-------------------------------+------+------+--------------+----------------------------------------------+-----------------------------------------+
|http://purl.obolibrary.org/obo/SO_0001583|[Not provided by data supplier]|pvalue|1.0E-7|null          |http://identifiers.org/ensembl/ENSG00000070915|http://www.orpha.net/ORDO/Orphanet_358   |
|http://purl.obolibrary.org/obo/SO_0001583|[Not provided by data supplier]|pvalue|1.0E-7|null          |http://identifiers.org/ensembl/ENSG00000123297|http://www.orpha.net/ORDO/Orphanet_168566|
|http://purl.obolibrary.org/ob